### Данный блокнот создан для классификации тестовых изображений и выгрузки результатов в требуемом формате для Kaggle.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from pathlib import Path

print('tensorflow version', tf.__version__)

tensorflow version 2.4.1


Функция для вырезания лица на основании предсказаний opencv_face_detector

In [2]:
# instantiate face detector
folder = 'face_detection_data/'
fd_net =  cv2.dnn.readNetFromCaffe(
                folder + 'opencv_face_detector.prototxt', 
                folder + 'opencv_face_detector.caffemodel'
            )

def get_faces(image):
    """
    Run inference of face detector and return list of boxes with faces.
    Image is ndarray in standard RGB format with 0-255 values for color.
    """
    (h, w) = image.shape[:2]
 
    # preprocess image, add batch dim, rearrange dimentions (batch,channels,h,w)
    # size, mean and scale are determined by face detector 
    # (https://github.com/opencv/opencv/blob/master/samples/dnn/models.yml)
    blob = cv2.dnn.blobFromImage(
        image=image, 
        scalefactor=1.0,
        size=(300, 300), 
        mean=(104.0, 177.0, 123.0),
        swapRB=True,
    )
    
    # run inference of face detector
    fd_net.setInput(blob)
    detections = fd_net.forward()
    
    # retrieve face boxes
    faces = []
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            faces.append(box.astype("int"))
    return faces

Функция для загрузки изображения и преобразования его по требуемому размеру с опцией вырезания лица, если модель обучалась на вырезанных лицах.

In [3]:
def load_and_process_image(filename, img_size=None, detector_flag=False):
    """
    Load test image and prepare it for classifier:
        - resize to img_size
        - trim image to face box as per face detector prediction
          It always takes the largest face.
          No cut if there's no face detected.
    Returns image as ndarray (h, w, 3), where channel values are in range [0, 1]
    """
    with Image.open(filename) as img:
        image = img.convert("RGB")
        
        if detector_flag:
            image_np = np.array(image)
            boxes = get_faces(image_np)
            # cut a face if we detect a face otherwise skip
            if len(boxes) != 0:
                # find box index with the largest area
                max_area = 0
                max_box_idx = 0
                for i, box in enumerate(boxes):
                    startW, startH, endW, endH = box
                    area = np.abs((endW - startW) * (endH - startH))
                    if area > max_area:
                        max_area = area
                        max_box_idx = i

                startW, startH, endW, endH = boxes[max_box_idx]
                face = image_np[startH:endH, startW:endW]
                image = Image.fromarray(face)
   
        image = image.resize((img_size, img_size))
        return np.array(image) / 255.


Функция создания csv файла. 

In [5]:
def generate_submition_file(model, img_size, detector_flag):
    """
    model - trained model
    img_size - size to resize images to
    detector_flag - True if model was trained on case2 dataset, otherwise - False
    """
    # labels are always the same so to cpu time, I hardcoded this dict
    labels_dict = {
        0: 'anger',
        1: 'contempt',
        2: 'disgust',
        3: 'fear',
        4: 'happy',
        5: 'neutral',
        6: 'sad',
        7: 'surprise',
        8: 'uncertain'
    }
    
    # get files in test_kaggle folder (assuming there's no extra files)
    files = [str(name) for  name in test_dir.glob('*')]
    
    # create DataFrame with classification results
    results = pd.DataFrame(files, columns=['path'])
    results['image_path'] = [x.split('/')[-1] for x in files] # filename only
    results['emotion'] = 'nan' # 'nan' is used for asserting later on
    BATCH_SIZE = 100
    i = 0
    print('Evaluating test data...\nSteps:')
    while i + BATCH_SIZE <= results.shape[0]:
        # construct batch manually reading filenames by amount of BATCH_SIZE 
        names = results.iloc[i:i+BATCH_SIZE]['path'].tolist()
        images = []
        for name in names:
            images.append(load_and_process_image(name, img_size, detector_flag))
        batch = np.array(images)
        
        # make prediction on a batch
        predictions = model(batch)
        y_pred = np.argmax(predictions, axis=-1)
        
        # write predicted class to DataFrame
        results.iloc[i:i+BATCH_SIZE, 2] = [labels_dict[x] for x in y_pred]
        i += BATCH_SIZE
        if i % 1000 == 0 and i > 0:
            print('  ', i)
    print('... Done.')
    
    # sort dataframe by filename
    results.sort_values('image_path', ascending=True, inplace=True)
    # export to csv as per requirements
    columns = ['image_path', 'emotion']
    results[columns].to_csv('results.csv', columns=columns, index=False)
    
    # simple assert that we didn't miss a file. 
    # It could happen if number of files is not multiple of BATCH_SIZE... I bit lazy to make it universal.
    assert not ('nan' in results.emotion.unique()), "some images left unprocessed/unevaluated..."
    print('results.csv was created.')
    return results

Протестируем модель

In [10]:
# set folder with test images
test_dir = Path('dataset') / 'test_kaggle'

# choose folder with saved model being tested
chpt_name = 'models/bit-m/chpt1'
# load model
model = tf.keras.models.load_model(chpt_name)
print(f"{chpt_name} checkpoint loaded.")

# test and export results
result_df = generate_submition_file(model, img_size=200, detector_flag=False)
result_df.head()

models/bit-m/chpt1 checkpoint loaded.
Evaluating test data...
Steps:
   1000
   2000
   3000
   4000
   5000
... Done.
results.csv was created.


,path,image_path,emotion
4634,dataset/test_kaggle/0.jpg,0.jpg,sad
3638,dataset/test_kaggle/1.jpg,1.jpg,neutral
2582,dataset/test_kaggle/10.jpg,10.jpg,neutral
2137,dataset/test_kaggle/100.jpg,100.jpg,uncertain
2902,dataset/test_kaggle/1000.jpg,1000.jpg,happy
